# Build a semantic search engine

Làm quen với document loader, embedding, vector store

## Content
1. Setup
2. LangSmith
3. Documents and Document Loaders
4. Loading documents
5. Splitting
6. Embeddings
7. Vector storess
8. Usage
9. Retrievers

    retrieval of data-- from (vector) databases and other sources

Ở đây chúng ta sẽ xây dựng một công cụ tìm kiếm trên một tài liệu PDF. Điều này sẽ cho phép chúng ta tìm kiếm các đoạn văn trong PDF tương tự như truy vấn đầu vào.


# 1. Setup

In [ ]:
%pip install langchain-community pypdf

# 2. LangSmith

Trong một hệ thống phức tạp chứa các bước gọi LLM, khi sự phức tạp ngày càng tăng, việc có thể kiểm tra chính xác hoặc tác nhân trở nên quan trọng hơn bao giờ hết. Cách tốt nhất để giám sát quá trình là sử dụng LangSmith.

- b1: truy cập https://smith.langchain.com/ (tạo tài khoản nếu cần thiết)
- b2: setup một trình giám sát tracing (set up tracing)
- b3: tạo api key tracing theo hướng dẫn và lưu nó
- b4: thiết lập biến môi trường cho các key api

In [ ]:
# For tracing
# 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

In [ ]:
import os
import getpass

os.environ["LANGSMITH_TRACING"] = "true"                        
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

# 3. Documents and Document Loaders

LangChain triển khai lớp trừu tượng Document để biểu diễn đơn vị văn bản và các metadata liên quan. Có 3 thuộc tính chính:
- page_content: Một chuỗi biểu diễn nội dung
- metadata: một dict chứa metadata tùy ý
- id (tùy chọn): Một chuỗi định dạng cho tài liệu

Chúng ta cũng có thể tạo Document ngay lập tức

In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"}
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"}
    )
]

# 4. Loading documents

Tuy nhiên chúng ta cũng có thể sử dụng các PDF loader trong các hệ thống của chúng ta

In [ ]:
import glob
from langchain_community.document_loaders import PyPDFLoader

In [5]:
folder_path = "../Datasets/Example_pdf_dataset/*"
list_pdf = glob.glob(folder_path)
list_pdf

['../Datasets/Example_pdf_dataset\\2404.10981v2.pdf',
 '../Datasets/Example_pdf_dataset\\2405.10098v1.pdf',
 '../Datasets/Example_pdf_dataset\\2405.13019v2.pdf',
 '../Datasets/Example_pdf_dataset\\2408.12599v1.pdf']

In [6]:
loader = PyPDFLoader(list_pdf[0])
docs = loader.load()
print(len(docs)) # Number of pages

37


Xem thêm tài liệu về PDF loader tại đây:

    https://python.langchain.com/docs/how_to/document_loader_pdf/

In [7]:
# The string content of the page;
# Metadata containing the file name and page number.
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

The Survey of Retrieval-Augmented Text Generation in Large
Language Models
YIZHENG HUANG and JIMMY X. HUANG, York University, Canada
Retrieval-Augmented Generation (RAG) merges retrieval methods with 

{'producer': 'pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'creator': 'LaTeX with acmart 2024/05/27 v2.08 Typesetting articles for the Association for Computing Machinery and hyperref 2023-04-22 v7.00x Hypertext links for LaTeX', 'creationdate': '2024-08-26T00:10:51+00:00', 'moddate': '2024-08-26T00:10:51+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '-  Computing methodologies  ->  Natural language generation.-  Information systems  ->  Information retrieval.', 'title': 'The Survey of Retrieval-Augmented Text Generation in Large Language Models', 'trapped': '/False', 'source': '../Datasets/Example_pdf_dataset\\2404.10981v2.pdf', 'total_pages': 37, 'page': 0, 'page_label'

# 5. Splitting

Đối với các tác vụ thực hiện truy xuất dữ liệu và trả về câu hỏi cuối cùng, 1 trang sẽ không mang lại nhiều ý nghĩa. Mục tiêu cuối cùng của chúng ta là trích xuất các đối tượng và trả lời câu hỏi đầu vào, và việc chia nhỏ PDF hơn nữa sẽ giúp đảm bảo rằng ý nghĩa của các phần có liên quan trong tài liệu không bị "làm mờ" bởi văn bản xung quanh.

Chúng ta sẽ chia văn bản thành các phần với 1000 ký tự và 200 ký tự chồng chéo ở mỗi phần (tức là 200 ký tự chồng đó là phần cuồi của phần trước và là phần đầu của phần sau, 2 phần liền kề sẽ chung một phần chồng chéo 200 ký tự đó)

    Xem thêm hướng dẫn tại đây, bao gồm cả việc trích xuất văn bản từ hình ảnh: https://python.langchain.com/docs/how_to/document_loader_pdf/

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True # Chúng ta có thể chỉ định vị trí start
)

all_splits = text_splitter.split_documents(docs)
print(len(all_splits))

190


# 6. Embeddings

Tìm kiếm vectơ là một cách phổ biến để lưu trữ và tìm kiếm trên dữ liệu phi cấu trúc (chẳng hạn như văn bản phi cấu trúc). Ý tưởng là lưu trữ các vectơ số được liên kết với văn bản. Với một truy vấn, chúng ta có thể nhúng nó dưới dạng một vectơ có cùng chiều và sử dụng các số liệu về độ tương tự vectơ (chẳng hạn như độ tương tự cosin) để xác định văn bản liên quan.

In [ ]:
%pip install -qU langchain-ollama

In [12]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")

In [ ]:
vector_0 = embeddings.embed_query(all_splits[0].page_content)
vector_1 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_0) == len(vector_1)
print(f'Generated vectors of length {len(vector_0)}\n')
print(vector_0[:10])

Văn bản được các mô hình embedding biểu diễn thành các vector, sau đó chúng ta lưu trữ các vector này vào một cấu trúc dữ liệu đặc biệt hỗ trợ tìm kiếm similarity hiệu quả.

=> Vector store ra đời!

# 7. Vector storess

Có nhiều nhà cung cấp kho lưu trữ vector database, trong dự án này sử dụng qdrant

In [ ]:
%pip install -qU langchain-qdrant

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")
vector_store = QdrantVectorStore(
    client=client,
    collection_name='test',
    embedding=embeddings,
)

In [ ]:
# Sau khi khởi tạo vector database, chúng ta có thể lập chỉ mục cho các tài liệu
ids = vector_store.add_documents(documents=all_splits)

# 8. Usage

Nhúng thường biểu diễn văn bản dưới dạng một vectơ "dày đặc" sao cho các văn bản có ý nghĩa tương tự gần nhau về mặt hình học. Điều này cho phép chúng ta lấy thông tin có liên quan chỉ bằng cách đưa vào một câu hỏi, mà không cần biết bất kỳ thuật ngữ khóa cụ thể nào được sử dụng trong tài liệu.

In [ ]:
# Return documents based on similarity to a string query:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)
print(results[0])

In [ ]:
# Async query
results = await vector_store.asimilarity_search("When was Nike incorporated?")
print(results[0])

In [ ]:
# Return scores
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

# 9. Retrievers

In [ ]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain

@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ]
)

phương thức as_retriever sẽ tạo ra một Retriever, cụ thể là một VectorStoreRetriever. Các retriever này bao gồm các thuộc tính search_type và search_kwargs cụ thể để xác định phương thức nào của vector store cơ bản cần gọi và cách tham số hóa chúng. Ví dụ, chúng ta có thể sao chép nội dung trên bằng cách sau:

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

VectorStoreRetriever hỗ trợ các loại tìm kiếm "similarity" (mặc định), "mmr" (mức độ liên quan tối đa, được mô tả ở trên) và "similarity_score_threshold". Chúng ta có thể sử dụng loại sau để ngưỡng các tài liệu đầu ra của trình thu thập theo điểm tương đồng. Trình thu thập có thể dễ dàng được tích hợp vào các ứng dụng phức tạp hơn, chẳng hạn như các ứng dụng tạo tăng cường thu thập (RAG) kết hợp một câu hỏi nhất định với ngữ cảnh đã thu thập thành lời nhắc cho LLM. Để tìm hiểu thêm về cách xây dựng ứng dụng như vậy, hãy xem hướng dẫn RAG. 